In [2]:
%%info

In [1]:
%%configure -f
{
"conf":{
    "spark.sql.extensions":"org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.catalog.glue_catalog":"org.apache.iceberg.spark.SparkCatalog",
    "spark.sql.catalog.glue_catalog.warehouse":"s3://chiholee-datalake001/transaction/iceberg/emr",
    "spark.sql.catalog.glue_catalog.catalog-impl":"org.apache.iceberg.aws.glue.GlueCatalog",
    "spark.sql.catalog.glue_catalog.io-impl":"org.apache.iceberg.aws.s3.S3FileIO",
    "spark.sql.catalog.glue_catalog.lock-impl":"org.apache.iceberg.aws.dynamodb.DynamoDbLockManager",
    "spark.sql.catalog.glue_catalog.lock.table":"myGlueLockTable"
    }
}

In [37]:
catalog_name = "glue_catalog"
bucket_name = "chiholee-datalake001"
database_name = "ecommerce"

table_name = "orders"
pk = 'order_id'
last_update_time = 'order_dt'

source_bucket_prefix = "transaction/cdc/raw"
source_path = f"s3://{bucket_name}/{source_bucket_prefix}"
source_table_name = table_name

iceberg_bucket_prefix = "transaction/iceberg/emr"
warehouse_path = f"s3://{bucket_name}/{iceberg_bucket_prefix}"
iceberg_table_name = f"{table_name}_cdc_emr_iceberg"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
print(f"{warehouse_path}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://chiholee-datalake001/transaction/iceberg/emr

In [5]:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder \
#     .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
#     .config(f"spark.sql.catalog.{catalog_name}.warehouse", f"{warehouse_path}") \
#     .config(f"spark.sql.catalog.{catalog_name}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog") \
#     .config(f"spark.sql.catalog.{catalog_name}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
#     .config("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
#     .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.sql import SparkSession
import boto3
from datetime import datetime



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
dynamodb = boto3.resource('dynamodb', region_name="ap-northeast-2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)

In [7]:
def create_table(dynamodb):
    try:
        table = dynamodb.create_table(
            TableName='ProcessedFiles',
            KeySchema=[
                {
                    'AttributeName': 'Id',  # 고정 파티션 키
                    'KeyType': 'HASH'
                }
            ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'Id',
                    'AttributeType': 'S'
                }
                # ProcessedDate는 여기서 정의할 필요 없음
            ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 1,
                'WriteCapacityUnits': 1
            }
        )
        # 테이블 생성 대기
        table.wait_until_exists()
        print("Table created successfully.")
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceInUseException':
            print("Table already exists.")
        else:
            raise


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from botocore.exceptions import ClientError
try:
    table = dynamodb.Table('ProcessedFiles')
    table.load()
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceNotFoundException':
        create_table(dynamodb)
        table = dynamodb.Table('ProcessedFiles')
    else:
        raise

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
def get_last_processed_info(table):
    try:
        response = table.get_item(
            Key={'Id': 'LastProcessedInfo'}
        )
        if 'Item' in response:
            # 파일 이름과 처리 날짜 둘 다 반환
            last_processed_date = datetime.strptime(response['Item']['LastProcessedDate'], '%Y-%m-%dT%H:%M:%S')
            return last_processed_date
        else:
            print("No last processed info found.")
            return None
    except ClientError as e:
        print(f"Failed to fetch last processed info: {e}")
        return None

def update_last_processed_info(table, start_date, last_processed_date):
    try:
        # DynamoDB 항목 업데이트
        response = table.update_item(
            Key={
                'Id': 'LastProcessedInfo'  # 변경: 'LastProcessedDate' -> 'LastProcessedInfo'
            },
            UpdateExpression="SET StartDate = :startdate, LastProcessedDate = :lastprocesseddate",
            ExpressionAttributeValues={
                ':startdate': start_date.isoformat(),
                ':lastprocesseddate': last_processed_date.isoformat()
            },
            ReturnValues="UPDATED_NEW"
        )
        print("Update succeeded:", response)
    except ClientError as e:
        print(f"Failed to update last processed info: {e}")
        
        
def generate_date_paths(start_date, end_date):
    current_date = start_date
    end_date = end_date.replace(hour=23, minute=59, second=59, microsecond=0)
    while current_date <= end_date:
        yield f"{source_path}/{database_name}/{source_table_name}/{current_date.strftime('%Y/%m/%d/')}"
        current_date += timedelta(days=1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
last_processed_date = get_last_processed_info(table)
start_date = datetime.now()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
print(last_processed_date)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2024-05-01 03:43:35

In [44]:
from datetime import datetime, timedelta

if last_processed_date:
#     paths = list(generate_date_paths(last_processed_date + timedelta(days=1), today))
    paths = list(generate_date_paths(last_processed_date, start_date))
else:
    # 마지막 처리 날짜 정보가 없을 경우, 고정 기간을 설정하거나 전체 데이터를 재처리
    paths = list(generate_date_paths(start_date - timedelta(days=30), start_date))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
print(paths)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['s3://chiholee-datalake001/transaction/cdc/raw/ecommerce/orders/2024/05/01/']

In [14]:
# df = spark.read.parquet("s3://chiholee-datalake001/transaction/cdc/raw/ecommerce/orders/2024/04/30/20240430-000106420.parquet")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
table

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dynamodb.Table(name='ProcessedFiles')

In [17]:
# paths = ['s3://chiholee-datalake001/transaction/cdc/raw/ecommerce/orders/2024/04/30/']

merged_df = None

for path in paths:
    try:
        df = spark.read.option("basePath", f'{source_path}/{database_name}/{source_table_name}/').parquet(path)
        if df.rdd.isEmpty():
            print(f"No data found at {path}")
        else:
            if merged_df is None:
                merged_df = df
                last_processed_file = path  # 첫 번째로 데이터가 있는 파일을 기록
            else:
                merged_df = merged_df.union(df)
            last_processed_file = path  # 최신 파일 경로를 업데이트
    except Exception as e:
        print(f"Error reading data from {path}: {e}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Error reading data from s3://chiholee-datalake001/transaction/cdc/raw/ecommerce/orders/2024/04/01/: [PATH_NOT_FOUND] Path does not exist: s3://chiholee-datalake001/transaction/cdc/raw/ecommerce/orders/2024/04/01.
Error reading data from s3://chiholee-datalake001/transaction/cdc/raw/ecommerce/orders/2024/04/02/: [PATH_NOT_FOUND] Path does not exist: s3://chiholee-datalake001/transaction/cdc/raw/ecommerce/orders/2024/04/02.
Error reading data from s3://chiholee-datalake001/transaction/cdc/raw/ecommerce/orders/2024/04/03/: [PATH_NOT_FOUND] Path does not exist: s3://chiholee-datalake001/transaction/cdc/raw/ecommerce/orders/2024/04/03.
Error reading data from s3://chiholee-datalake001/transaction/cdc/raw/ecommerce/orders/2024/04/04/: [PATH_NOT_FOUND] Path does not exist: s3://chiholee-datalake001/transaction/cdc/raw/ecommerce/orders/2024/04/04.
Error reading data from s3://chiholee-datalake001/transaction/cdc/raw/ecommerce/orders/2024/04/05/: [PATH_NOT_FOUND] Path does not exist: s3://chiho

In [18]:
# merged_df.show()
# last_processed_date

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# if merged_df:
#     merged_df.show()  # 병합된 데이터셋 출력
#     update_last_processed_date(table, start_date, last_processed_date)  # DynamoDB에 마지막으로 처리된 날짜 업데이트

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
print(last_processed_date)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

None

In [21]:
merged_df.createOrReplaceTempView("cdcDf")

# 쿼리 작성 로직 조정
if last_processed_date is not None:
    # last_processed_date를 datetime 객체로 변환 (이미 datetime 형태일 수 있음)
    if isinstance(last_processed_date, str):
        last_processed_date = datetime.strptime(last_processed_date, '%Y-%m-%dT%H:%M:%S.%f')
    # last_processed_date에서 1분을 빼기
    time_threshold = last_processed_date - timedelta(minutes=1)
    # 날짜 조건이 있는 쿼리
    query = f"""
    select *
    from cdcDf
    where (order_id, order_dt) in
    (
        select order_id, max(order_dt) max_op_time
        from cdcDf
        WHERE order_dt > '{time_threshold.strftime('%Y-%m-%d %H:%M:%S')}'
        group by order_id
    )
    """
else:
    # 날짜 조건이 없는 쿼리
    query = """
    select *
    from cdcDf
    where (order_id, order_dt) in
    (
        select order_id, max(order_dt) max_op_time
        from cdcDf
        group by order_id
    )
    """

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
cdcDf = spark.sql(query)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
cdcInsertCount = cdcDf.filter("Op = 'I'").count()
cdcUpdateCount = cdcDf.filter("Op = 'U'").count()
cdcDeleteCount = cdcDf.filter("Op = 'D'").count()
print(f"Inserted count: {cdcInsertCount}")
print(f"Updated count: {cdcUpdateCount}")
print(f"Deleted count: {cdcDeleteCount}")
print(f"Total CDC count: {cdcDf.count()}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Inserted count: 286836
Updated count: 31849
Deleted count: 0
Total CDC count: 318685

In [24]:
dropColumnList = ['Op','dms_update_time']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
from datetime import datetime
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import concat, col, lit, to_timestamp

current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
cdcDf = cdcDf.withColumn('order_dt',to_timestamp(col('order_dt')))
cdcDf = (cdcDf
      .withColumn('year', year(col('order_dt')))
      .withColumn('month', month(col('order_dt')))
      .withColumn('day', dayofmonth(col('order_dt')))
     )
cdcDf = cdcDf.withColumn('last_applied_date',to_timestamp(lit(current_datetime)))



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {catalog_name}.{database_name}")
# existing_tables = spark.sql(f"SHOW TABLES IN {catalog_name}.{database_name};")
# df_existing_tables = existing_tables.select('tableName').rdd.flatMap(lambda x:x).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [27]:
upsertDf = cdcDf.filter("Op != 'D'").drop(*dropColumnList)
upsertDf.createOrReplaceTempView(f"{source_table_name}_upsert")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
spark.sql(f"""
select order_id, count(*)
from {catalog_name}.{database_name}.{iceberg_table_name}
group by order_id
having count(*) > 1
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+
|order_id|count(1)|
+--------+--------+
+--------+--------+

In [32]:
print(f"Table {source_table_name}_iceberg is upserting...")
spark.sql(f"""MERGE INTO {catalog_name}.{database_name}.{iceberg_table_name} t
    USING {source_table_name}_upsert s ON s.{pk} = t.{pk}
    WHEN MATCHED THEN UPDATE SET *
    WHEN NOT MATCHED THEN INSERT *
    """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o94.sql.
: org.apache.spark.SparkException: Job aborted due to stage failure: Authorized committer (attemptNumber=0, stage=94, partition=11) failed; but task commit success, data duplication may happen. reason=ExceptionFailure(java.io.UncheckedIOException,Failed to create output stream for location: s3://chiholee-datalake001/transaction/iceberg/emr/ecommerce.db/orders_cdc_emr_iceberg/data/year=2024/month=4/day=27/00011-5729-f7bc2aa8-0240-4c45-ad21-4809f2e98cd8-00001.parquet,[Ljava.lang.StackTraceElement;@18e913b9,java.io.UncheckedIOException: Failed to create output stream for location: s3://chiholee-datalake001/transaction/iceberg/emr/ecommerce.db/orders_cdc_emr_iceberg/data/year=2024/month=4/day=27/00011-5729-f7bc2aa8-0240-4c45-ad21-4809f2e98cd8-00001.parquet
	at org.apache.iceberg.aws.s3.S3OutputFile.createOrOverwrite(S3OutputFile.java:72)
	at org.apache.iceberg.parquet.ParquetIO$ParquetOutputFile.createOrOverwrite(ParquetIO.

In [34]:
upsertDf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+---------+-----------+-------------------+-----------+----------+----+-----+---+-------------------+
|order_id|promo_id|order_cnt|order_price|           order_dt|customer_id|product_id|year|month|day|  last_applied_date|
+--------+--------+---------+-----------+-------------------+-----------+----------+----+-----+---+-------------------+
|  290165| PROMO03|        8|       9000|2024-04-24 15:28:52|         60|         4|2024|    4| 24|2024-05-01 03:14:39|
|  134566| PROMO19|        7|      14000|2024-04-24 15:28:55|         89|         3|2024|    4| 24|2024-05-01 03:14:39|
|  290166| PROMO10|        6|      34000|2024-04-24 15:28:56|         81|        18|2024|    4| 24|2024-05-01 03:14:39|
|  290167| PROMO14|        9|      44000|2024-04-24 15:28:57|         49|         3|2024|    4| 24|2024-05-01 03:14:39|
|  290168| PROMO12|       10|      13000|2024-04-24 15:28:57|         99|        13|2024|    4| 24|2024-05-01 03:14:39|
|  290169| PROMO13|        5|      12000

In [35]:
max_order_dt = upsertDf.agg({"order_dt": "max"}).collect()[0]["max(order_dt)"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
print(max_order_dt)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2024-05-01 03:12:11

In [ ]:
update_last_processed_date(table, start_date, max_order_dt)